In [1]:
from difflib import SequenceMatcher
import pandas as pd
import numpy as np
from thefuzz import fuzz
from thefuzz import process
from numpy import linalg as LA
import matplotlib.pyplot as plt
import plotly.express as px
from joblib import Parallel, delayed
from random import choices
from random import choice
from math import radians, sin, cos, sqrt, atan2

# import pyproj



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [14]:
# jellyfish.levenshtein_distance('jellyfish', 'smellyfish')
# jellyfish.jaro_distance('jellyfish', 'smellyfish')
# jellyfish.damerau_levenshtein_distance('jellyfish', 'jellyfihs')
# jellyfish.metaphone('Jellyfish')
# jellyfish.soundex('Jellyfish')
# jellyfish.nysiis('Jellyfish')
# jellyfish.match_rating_codex('Jellyfish')

'JLYFSH'

In [3]:
df = pd.read_csv('data/Person_data_1mil.csv')
df_HH = pd.read_csv('data/HH_data_1mil.csv')

/Users/muhammadfathifadlian/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/muhammadfathifadlian/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
geocw = pd.read_csv('data/geoCrosswalk/GeoCrossWalk.csv')
ISO_list = ['GBR', 'BEL', 'AUT', 'DEU', 'FRA', 'ESP', 'ITA', 'CHE', 'DNK', 'PRT', 'NLD']
geocw = geocw[geocw['HH_ISO'].isin(ISO_list)]

In [5]:
df_use = df[df['HH_ISO'].isin(ISO_list)]

In [278]:
sample_size_per_nationality = 3000

# Create an empty DataFrame to store the sampled data
sampled_df = pd.DataFrame(columns=df_use.columns)

# Group the DataFrame by 'nationality'
grouped = df_use.groupby('HH_ISO')

# Iterate through each group and pick random samples
for Country, group_data in grouped:
    if len(group_data) >= sample_size_per_nationality:
        # If the group has enough data points, pick random samples
        random_samples = group_data.sample(sample_size_per_nationality, random_state=42)
        sampled_df = pd.concat([sampled_df, random_samples])
    else:
        # If the group has fewer data points than the sample size, include all of them
        sampled_df = pd.concat([sampled_df, group_data])
sampled_df

,Unnamed: 0,P_num,HHID,P_ID,P_AGE,AgeRange,AgeGroup,P_GENDER,HH_ISO,GenderHOH,...,DOB,Address,PostCode,Country,FreeEmail,WorkEmail,DOCS_ExpiryDate,PaymentInfo_VendorCode,PaymentInfo_ExpiryDate,PaymentInfo_AccountNbr
33889,33889,33890,AUT_14286,AUT_14286_1,68,65-69,AGE14,M,AUT,M,...,1953-10-06,Laetitia-Ortner-Straße 05\n6305 Neulengbach,1683,Austria,albert79@yahoo.co.uk,sarahpatel@davies-wells.org,2022-01-06,American Express,2023-09-20,4218318178219529
37224,37224,37225,AUT_15654,AUT_15654_2,37,35-39,AGE8,M,AUT,F,...,1984-08-14,Lucas-Gratzl-Weg 88\n7299 Hainburg an der Donau,5954,Austria,mknight@gmail.com,zturner@george.com,2026-06-14,VISA 19 digit,2023-02-08,5516936806179751
27269,27269,27270,AUT_11517,AUT_11517_2,62,60-64,AGE13,F,AUT,M,...,1959-10-20,Wilhelmgasse 4\n1859 Pregarten,1440,Austria,pphillips@yahoo.com,wbennett@ellis.com,2025-07-13,JCB 15 digit,2026-11-15,6011648090902792
21593,21593,21594,AUT_9103,AUT_9103_3,58,55-59,AGE12,F,AUT,M,...,1963-03-08,Geyerring 6\n9060 Trieben,1489,Austria,hwilson@yahoo.co.uk,dawn38@taylor-davidson.org,2023-08-05,JCB 16 digit,2024-02-07,180005143920529
36422,36422,36423,AUT_15319,AUT_15319_1,74,70-74,AGE15,M,AUT,M,...,1947-02-25,Schöpfstraße 01\n3005 Ternitz,9861,Austria,simonhorton@yahoo.co.uk,jodiemclean@holland-smith.com,2022-08-05,VISA 16 digit,2023-06-15,30206505346202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704039,1704039,1704040,PRT_13888,PRT_13888_2,51,50-54,AGE11,F,PRT,M,...,1970-02-01,"Praça Sandro Ferreira, 5\n1082-992 Alcobaça",4719-983,Portugal,atkinsondenise@hotmail.com,lewissaunders@clarke-bowen.com,2026-11-06,Discover,2022-08-21,060448112734
1708909,1708909,1708910,PRT_15716,PRT_15716_2,74,70-74,AGE15,M,PRT,F,...,1947-07-14,"R. Manuel Jesus, 6\n6760-723 São Mamede de Inf...",0990-768,Portugal,patrickfletcher@yahoo.co.uk,qwilson@gibbs.org,2023-12-01,Maestro,2025-07-09,4690537501491300
1698408,1698408,1698409,PRT_11765,PRT_11765_1,43,40-44,AGE9,M,PRT,M,...,1978-10-04,"Avenida Marco Simões, S/N\n6516-441 Felgueiras",1043-489,Portugal,elainemarshall@yahoo.com,taylorkaty@taylor-harvey.co.uk,2024-12-06,JCB 16 digit,2023-05-19,4222107559438262894
1683288,1683288,1683289,PRT_6100,PRT_6100_1,51,50-54,AGE11,M,PRT,M,...,1970-01-10,"Praça do Preto, 408\n2682-437 Mêda",4461-662,Portugal,arnoldleonard@yahoo.com,slyons@phillips.org,2023-08-27,JCB 16 digit,2023-03-08,502092184840


In [70]:
sampled_df.groupby('P_GENDER').count()

,Unnamed: 0,P_num,HHID,P_ID,P_AGE,AgeRange,AgeGroup,HH_ISO,GenderHOH,AgeHOH,...,DOB,Address,PostCode,Country,FreeEmail,WorkEmail,DOCS_ExpiryDate,PaymentInfo_VendorCode,PaymentInfo_ExpiryDate,PaymentInfo_AccountNbr
P_GENDER,,,,,,,,,,,,,,,,,,,,,
F,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20
M,35,35,35,35,35,35,35,35,35,35,...,35,35,35,35,35,35,35,35,35,35


In [7]:
def assignCity(geocw, df_use):
    par = df_use['HH_ISO']
    df_city_lookup = pd.DataFrame(columns=['HH_ISO', 'City', 'airportID'])
    df_handler = pd.DataFrame(columns=['HH_ISO', 'City_O', 'Cap_Cit'])
    for HH_ISO in df_use['HH_ISO'].unique():
        org = geocw.groupby(['HH_ISO', 'City'])['airportID'].count().reset_index()
        org = org[org['HH_ISO'] == HH_ISO]
        cities = list(org['City'])
        cap_cit = list(org['airportID']/org['airportID'].sum())
        df_handler['HH_ISO'] = pd.Series(HH_ISO)
        df_handler['City_O'] = pd.Series([cities])
        df_handler['Cap_Cit'] = pd.Series([cap_cit])
        df_city_lookup = df_city_lookup.append(df_handler)
        
    a = Parallel(n_jobs=6)(delayed(chooseCity)(i, df_city_lookup) for i in par)
    city = zip(*a)
    city = [item for sublist in city for item in sublist]
    df_use['City'] = city
    
    df_geo = geocw[['City', 'Longitude', 'Latitude']].copy()
    df_geo = df_geo.drop_duplicates(subset = ['City'])
    df_use = df_use[['P_num', 'P_ID', 'P_AGE', 'P_GENDER', 'HH_ISO', 'Surname', 'FirstName', 'DOB', 'City']].copy()
    df_result = pd.merge(df_use, df_geo, on=['City'], how = 'left')
    return df_result
        
        
def chooseCity(i, df_city_lookup):
    x = choices(df_city_lookup[df_city_lookup['HH_ISO'] == i]['City_O'][0], 
                df_city_lookup[df_city_lookup['HH_ISO'] == i]['Cap_Cit'][0])
    return x

In [279]:
df_result = assignCity(geocw, sampled_df)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [280]:
duplicated_rows = pd.concat([df_result] * 3, ignore_index=True)

In [10]:
duplicated_rows

,P_num,P_ID,P_AGE,P_GENDER,HH_ISO,Surname,FirstName,DOB,City,Longitude,Latitude
0,33890,AUT_14286_1,68,M,AUT,Graf,Nathan,1953-10-06,Graz,15.439600,46.991100
1,37225,AUT_15654_2,37,M,AUT,Siller,Samuel,1984-08-14,Graz,15.439600,46.991100
2,27270,AUT_11517_2,62,F,AUT,Nöbauer,Bethan,1959-10-20,Graz,15.439600,46.991100
3,21594,AUT_9103_3,58,F,AUT,Krainer,Marilyn,1963-03-08,Graz,15.439600,46.991100
4,36423,AUT_15319_1,74,M,AUT,Walder,Louis,1947-02-25,Graz,15.439600,46.991100
...,...,...,...,...,...,...,...,...,...,...,...
160,1712533,PRT_17137_1,72,M,PRT,Morais,Marcus,1949-03-27,Faro,-7.965910,37.014400
161,1687205,PRT_7576_1,34,F,PRT,Pires,Jodie,1987-01-08,Santa Maria (island),-25.170601,36.971401
162,1698802,PRT_11914_1,72,M,PRT,Loureiro,Jack,1949-01-31,Faro,-7.965910,37.014400
163,1671054,PRT_1525_3,1,F,PRT,Gonçalves,Kelly,2020-04-22,Santa Maria (island),-25.170601,36.971401


In [142]:
data = df_use['name'].tolist()
data = data[0:3000]

In [11]:
import random
from datetime import datetime, timedelta

def introduce_typos(text, typo_rate):
    typo_text = list(text)
    for i in range(len(typo_text)-1):
        if random.random() < typo_rate:
            # Introduce a typo (e.g., swap with the next character)
            typo_text[i], typo_text[i+1] = typo_text[i+1], typo_text[i]
    return ''.join(typo_text)



def introduce_dob_typos(dob, typo_rate):
    try:
        dob_date = datetime.strptime(dob, '%Y-%m-%d')  # Original format is 'Year-Month-Date'
        
        # Determine whether to introduce a typo
        if random.random() < typo_rate:
            # Generate random offsets for year, month, and day
            year_offset = random.randint(-10, 10)  # Allow typos within a 10-year range
            month_offset = random.randint(-6, 6)  # Allow typos within a 6-month range
            day_offset = random.randint(-15, 15)  # Allow typos within a 15-day range
            
            # Apply offsets to the date
            typo_year = dob_date.year + year_offset
            typo_month = dob_date.month + month_offset
            typo_day = dob_date.day + day_offset
            
            # Handle edge cases where the month or day might go out of bounds
            while typo_month <= 0:
                typo_month += 12
                typo_year -= 1
            while typo_month > 12:
                typo_month -= 12
                typo_year += 1
            while typo_day <= 0:
                typo_month -= 1
                typo_day += (30 if typo_month in {4, 6, 9, 11} else 31 if typo_month != 2 else 29 if typo_year % 4 == 0 and (typo_year % 100 != 0 or typo_year % 400 == 0) else 28)
            while typo_day > (30 if typo_month in {4, 6, 9, 11} else 31 if typo_month != 2 else 29 if typo_year % 4 == 0 and (typo_year % 100 != 0 or typo_year % 400 == 0) else 28):
                typo_month += 1
                typo_day -= (30 if typo_month in {4, 6, 9, 11} else 31 if typo_month != 2 else 29 if typo_year % 4 == 0 and (typo_year % 100 != 0 or typo_year % 400 == 0) else 28)
            
            # Create a new date with the typos
            typo_dob_date = datetime(typo_year, typo_month, typo_day)
            
            # Format the typo DOB back to the original format
            typo_dob = typo_dob_date.strftime('%Y-%m-%d')
            return typo_dob
    except ValueError:
        # Handle invalid DOB formats gracefully (e.g., if the original DOB is already incorrect)
        pass
    
    return dob  # Return the original DOB if no typo is introduced or if there's an issue with the format



In [12]:
def introduce_dob_typos2(dob, typo_rate):
    dob_parts = dob.split("-")
    old_dob = dob
    year = dob_parts[0]
    month = dob_parts[1]
    day = dob_parts[2]
    
    if random.random() < typo_rate:
        year = list(year)
        digit_to_change = random.randint(0, len(year) - 1)
        year[digit_to_change] = str(random.randint(0, 9))
        year = "".join(year)

    if random.random() < typo_rate:
        month = list(month)
        digit_to_change = random.randint(0, len(month) - 1)
        month[digit_to_change] = str(random.randint(0, 1))
        month = "".join(month)

    if random.random() < typo_rate:
        day = list(day)
        digit_to_change = random.randint(0, len(day) - 1)
        day[digit_to_change] = str(random.randint(0, 3))
        day = "".join(day)
        
    try:
        new_date = datetime(int(year), int(month), int(day))
    except ValueError:
        # Invalid date, correct it to a valid date
        year = dob_parts[0]
        month = dob_parts[1]
        day = dob_parts[2]
    if  (int(year) > 2100 or int(year) < 1900):
        year = dob_parts[0]

    return f"{year}-{month}-{day}"

In [13]:
def generate_random_coordinates_variation(row, min_percentage=0.01, max_percentage=0.05):
    # Generate random percentages for both longitude and latitude
    longitude_percentage = random.uniform(min_percentage, max_percentage)
    latitude_percentage = random.uniform(min_percentage, max_percentage)

    # Calculate the random variations
    longitude_variation = row['Longitude'] * longitude_percentage
    latitude_variation = row['Latitude'] * latitude_percentage

    # Apply the variations to the original values
    new_longitude = row['Longitude'] + random.choice([-1, 1]) * longitude_variation
    new_latitude = row['Latitude'] + random.choice([-1, 1]) * latitude_variation

    return pd.Series({'Longitude': new_longitude, 'Latitude': new_latitude})

In [14]:
def normalize_datetime(df, datetime_column, start_date, end_date):
    """
    df: DataFrame
    datetime_column: Column containing datetime values
    start_date: Start of the original time range
    end_date: End of the original time range
    """
    
    df["DT_Temp"] = pd.to_datetime(df[datetime_column])
    # Calculate timestamps for the start and end dates
    timestamp_start = 0
    timestamp_end = (end_date - start_date).total_seconds()

    # Calculate timestamps for the datetime column
    df['timestamp'] = (df["DT_Temp"] - start_date).dt.total_seconds()

    # Normalize the timestamps to a range from -1 to 1
    df['normalized_timestamp'] = ((df['timestamp'] - timestamp_start) / (timestamp_end - timestamp_start)) * 2 - 1

    # Drop the temporary 'timestamp' column
    df = df.drop(columns=['timestamp'])
    df = df.drop(columns=["DT_Temp"])

    return df

In [15]:
def convert_to_mercator(df, lon_col, lat_col):
    """
    df: DataFrame
    lon_col: Name of the column containing longitude coordinates
    lat_col: Name of the column containing latitude coordinates
    Returns a DataFrame with Mercator coordinates added as 'x' and 'y' columns.
    """
    # Define the source and target projections (WGS 84 to Mercator)
    wgs84 = pyproj.Proj(init="epsg:4326")
    mercator = pyproj.Proj(init="epsg:3857")

    # Convert longitude and latitude coordinates to Mercator coordinates
    df['x'], df['y'] = pyproj.transform(wgs84, mercator, df[lon_col].tolist(), df[lat_col].tolist())

    return df

In [281]:
duplicated_rows['DOB'] = duplicated_rows['DOB'].apply(lambda x: introduce_dob_typos2(x, typo_rate=0.2))
duplicated_rows['FirstName'] = duplicated_rows['FirstName'].apply(lambda x: introduce_typos(x, typo_rate=0.2))
duplicated_rows['Surname'] = duplicated_rows['Surname'].apply(lambda x: introduce_typos(x, typo_rate=0.2))
modified_coordinates = duplicated_rows.apply(generate_random_coordinates_variation, axis=1)
duplicated_rows[['Longitude', 'Latitude']] = modified_coordinates[['Longitude', 'Latitude']]

In [17]:
duplicated_rows

,P_num,P_ID,P_AGE,P_GENDER,HH_ISO,Surname,FirstName,DOB,City,Longitude,Latitude
0,33890,AUT_14286_1,68,M,AUT,Graf,Nahatn,1953-10-06,Graz,14.963946,49.050156
1,37225,AUT_15654_2,37,M,AUT,Silelr,Sameul,1984-08-14,Graz,15.652221,44.689354
2,27270,AUT_11517_2,62,F,AUT,Nöabure,Bthaen,1959-10-20,Graz,15.614616,49.079168
3,21594,AUT_9103_3,58,F,AUT,rKainre,Marilyn,1963-03-08,Graz,14.863890,44.831277
4,36423,AUT_15319_1,74,M,AUT,aWlder,Louis,1947-02-21,Graz,14.922678,45.933905
...,...,...,...,...,...,...,...,...,...,...,...
160,1712533,PRT_17137_1,72,M,PRT,Morais,Macrsu,1949-03-27,Faro,-8.319535,38.217210
161,1687205,PRT_7576_1,34,F,PRT,iPres,Jdoei,1989-01-08,Santa Maria (island),-26.178422,35.652486
162,1698802,PRT_11914_1,72,M,PRT,ouLriero,Jack,1949-01-31,Faro,-7.829808,36.248437
163,1671054,PRT_1525_3,1,F,PRT,Gonçavles,Kelly,2020-04-12,Santa Maria (island),-24.360267,37.746097


In [282]:
frames = [df_result, duplicated_rows]
df_combined = pd.concat(frames)
df_combined

,P_num,P_ID,P_AGE,P_GENDER,HH_ISO,Surname,FirstName,DOB,City,Longitude,Latitude
0,33890,AUT_14286_1,68,M,AUT,Graf,Nathan,1953-10-06,Vienna,16.569700,48.110298
1,37225,AUT_15654_2,37,M,AUT,Siller,Samuel,1984-08-14,Vienna,16.569700,48.110298
2,27270,AUT_11517_2,62,F,AUT,Nöbauer,Bethan,1959-10-20,Vienna,16.569700,48.110298
3,21594,AUT_9103_3,58,F,AUT,Krainer,Marilyn,1963-03-08,Vienna,16.569700,48.110298
4,36423,AUT_15319_1,74,M,AUT,Walder,Louis,1947-02-25,Vienna,16.569700,48.110298
...,...,...,...,...,...,...,...,...,...,...,...
98995,1704040,PRT_13888_2,51,F,PRT,atiBsta,Jade,1970-02-01,Ponta Delgada,-24.953509,36.658229
98996,1708910,PRT_15716_2,74,M,PRT,Teixirea,Adam,1947-07-14,Santa Maria (island),-26.412746,38.693169
98997,1698409,PRT_11765_1,43,M,PRT,Castro,Braldey,1978-10-04,Ponta Delgada,-26.104885,39.392809
98998,1683289,PRT_6100_1,51,M,PRT,oCsta,Henyr,1970-01-10,Faro,-8.053168,38.808802


In [283]:
start_date = datetime(1900, 1, 1)
end_date = datetime(2100, 12, 31)
df_combined = normalize_datetime(df_combined, 'DOB', start_date, end_date)
df_combined

,P_num,P_ID,P_AGE,P_GENDER,HH_ISO,Surname,FirstName,DOB,City,Longitude,Latitude,normalized_timestamp
0,33890,AUT_14286_1,68,M,AUT,Graf,Nathan,1953-10-06,Vienna,16.569700,48.110298,-0.465054
1,37225,AUT_15654_2,37,M,AUT,Siller,Samuel,1984-08-14,Vienna,16.569700,48.110298,-0.158024
2,27270,AUT_11517_2,62,F,AUT,Nöbauer,Bethan,1959-10-20,Vienna,16.569700,48.110298,-0.404983
3,21594,AUT_9103_3,58,F,AUT,Krainer,Marilyn,1963-03-08,Vienna,16.569700,48.110298,-0.371338
4,36423,AUT_15319_1,74,M,AUT,Walder,Louis,1947-02-25,Vienna,16.569700,48.110298,-0.530846
...,...,...,...,...,...,...,...,...,...,...,...,...
98995,1704040,PRT_13888_2,51,F,PRT,atiBsta,Jade,1970-02-01,Ponta Delgada,-24.953509,36.658229,-0.302630
98996,1708910,PRT_15716_2,74,M,PRT,Teixirea,Adam,1947-07-14,Santa Maria (island),-26.412746,38.693169,-0.527059
98997,1698409,PRT_11765_1,43,M,PRT,Castro,Braldey,1978-10-04,Ponta Delgada,-26.104885,39.392809,-0.216351
98998,1683289,PRT_6100_1,51,M,PRT,oCsta,Henyr,1970-01-10,Faro,-8.053168,38.808802,-0.303230


In [284]:
df_combined['GENDER_BIN'] = np.where(df_combined['P_GENDER'] == "M", 1, -1)
df_combined

,P_num,P_ID,P_AGE,P_GENDER,HH_ISO,Surname,FirstName,DOB,City,Longitude,Latitude,normalized_timestamp,GENDER_BIN
0,33890,AUT_14286_1,68,M,AUT,Graf,Nathan,1953-10-06,Vienna,16.569700,48.110298,-0.465054,1
1,37225,AUT_15654_2,37,M,AUT,Siller,Samuel,1984-08-14,Vienna,16.569700,48.110298,-0.158024,1
2,27270,AUT_11517_2,62,F,AUT,Nöbauer,Bethan,1959-10-20,Vienna,16.569700,48.110298,-0.404983,-1
3,21594,AUT_9103_3,58,F,AUT,Krainer,Marilyn,1963-03-08,Vienna,16.569700,48.110298,-0.371338,-1
4,36423,AUT_15319_1,74,M,AUT,Walder,Louis,1947-02-25,Vienna,16.569700,48.110298,-0.530846,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98995,1704040,PRT_13888_2,51,F,PRT,atiBsta,Jade,1970-02-01,Ponta Delgada,-24.953509,36.658229,-0.302630,-1
98996,1708910,PRT_15716_2,74,M,PRT,Teixirea,Adam,1947-07-14,Santa Maria (island),-26.412746,38.693169,-0.527059,1
98997,1698409,PRT_11765_1,43,M,PRT,Castro,Braldey,1978-10-04,Ponta Delgada,-26.104885,39.392809,-0.216351,1
98998,1683289,PRT_6100_1,51,M,PRT,oCsta,Henyr,1970-01-10,Faro,-8.053168,38.808802,-0.303230,1


In [86]:
duplicated_rows['name'] = duplicated_rows['FirstName']+" "+duplicated_rows['Surname']

In [92]:
data = duplicated_rows['name'].tolist()
data = data[55:]
n = len(data)

aff_mat = np.zeros((n,n)) # This is the S matrix in the paper
D = np.zeros((n,n))

for ii in range(n):
    for jj in range(n):
        name1 = data[ii]
        name2 = data[jj]
    
        surname1 = name1.split()[0]
        lastname1 = name1.split()[1]
    
        surname2 = name2.split()[0]
        lastname2 = name2.split()[1]
    
        aff_name1_name2 = fuzz.ratio(surname1,surname2) + fuzz.ratio(lastname1,lastname2)
#         aff_name1_name2 = jellyfish.jaro_distance(surname1,surname2) + jellyfish.jaro_distance(lastname1,lastname2)

        # Fuzz ratios are betweein 0 and 100 and we add 2 of them 
        # so we normalize the whole score to 0 and 1 by dividing by 200
        aff_mat[ii,jj] = aff_name1_name2/200
    


In [93]:

# calculate degree matrix (sum of affinities)
for ii in range(n):
    D[ii,ii] = np.sum(aff_mat[:,ii])
L = D - aff_mat # Laplacian matrix

# compute eigenvector decomposition (eigenvectors, eigenvalues) of L
evals, evcts = LA.eig(L)

# extract "smallest" 3 eigenvectors (ignoring first one)
sortedevals = np.argsort(evals)
U = evcts[:,sortedevals[1:4]] # projection into lower-dimensional space



In [622]:
df2 = pd.DataFrame(U, columns = ['x', 'y', 'z'])
# df2 = pd.DataFrame(U, columns = ['x', 'y'])
df2['name'] = data
df2

fig = px.scatter_3d(df2, x="x", y="y", z = 'z', text="name")
# fig = px.scatter(df2, x="x", y="y", text="name")
fig.update_layout(yaxis_range=[-1,1])
fig.update_layout(xaxis_range=[-1,1])
# fig.update_layout(zaxis_range=[-1,1])


fig

In [95]:
new_surname = "Nathan"
new_lastname = "Graaf"

# Initialize a list to store similarity scores
similarity_scores = []

# Calculate similarity with each existing pair
for name1 in data:
    surname1 = name1.split()[0]
    lastname1 = name1.split()[1]
    affinity_score = fuzz.ratio(new_surname, surname1) + fuzz.ratio(new_lastname, lastname1)
    normalized_score = affinity_score / 200
    similarity_scores.append(normalized_score)

# Define a similarity threshold
threshold = 0.6  # Adjust as needed

# Check if the new pair is similar to any existing pair
similar_pairs = [i for i, score in enumerate(similarity_scores) if score >= threshold]

# If similar_pairs is not empty, the new pair is considered similar to the existing pairs.
if similar_pairs:
    print("The new pair is similar to the following existing pairs:", similar_pairs)
else:
    print("The new pair is not similar to any existing pairs.")

The new pair is similar to the following existing pairs: [0, 55]


In [100]:
similarity_scores

[0.835,
 0.175,
 0.335,
 0.32,
 0.09,
 0.11,
 0.235,
 0.28,
 0.155,
 0.1,
 0.155,
 0.255,
 0.09,
 0.27,
 0.17,
 0.155,
 0.28,
 0.18,
 0.09,
 0.325,
 0.245,
 0.23,
 0.2,
 0.29,
 0.2,
 0.0,
 0.32,
 0.375,
 0.2,
 0.325,
 0.18,
 0.07,
 0.215,
 0.16,
 0.19,
 0.23,
 0.24,
 0.35,
 0.075,
 0.075,
 0.335,
 0.24,
 0.09,
 0.275,
 0.25,
 0.33,
 0.225,
 0.255,
 0.22,
 0.1,
 0.265,
 0.1,
 0.175,
 0.145,
 0.265,
 0.635,
 0.175,
 0.335,
 0.24,
 0.09,
 0.11,
 0.16,
 0.28,
 0.155,
 0.1,
 0.155,
 0.255,
 0.09,
 0.27,
 0.17,
 0.155,
 0.28,
 0.18,
 0.09,
 0.325,
 0.165,
 0.23,
 0.2,
 0.29,
 0.2,
 0.0,
 0.32,
 0.31,
 0.2,
 0.325,
 0.18,
 0.07,
 0.215,
 0.16,
 0.19,
 0.23,
 0.24,
 0.265,
 0.075,
 0.075,
 0.335,
 0.24,
 0.09,
 0.275,
 0.25,
 0.25,
 0.225,
 0.255,
 0.22,
 0.1,
 0.265,
 0.1,
 0.175,
 0.145,
 0.265]

In [101]:
affinity_vector = np.array(similarity_scores).reshape(1, -1)

In [102]:
L_combined = np.block([[L, affinity_vector.T], [affinity_vector, 0]])

In [145]:
L

array([[23.095, -0.185, -0.34 , ..., -0.185, -0.155, -0.19 ],
       [-0.185, 25.765, -0.24 , ..., -0.245, -0.315, -0.075],
       [-0.34 , -0.24 , 26.635, ..., -0.3  , -0.215, -0.17 ],
       ...,
       [-0.185, -0.245, -0.3  , ..., 24.465, -0.12 , -0.19 ],
       [-0.155, -0.315, -0.215, ..., -0.12 , 22.785, -0.125],
       [-0.19 , -0.075, -0.17 , ..., -0.19 , -0.125, 19.26 ]])

In [144]:
L_combined

array([[23.095, -0.185, -0.34 , ..., -0.155, -0.19 ,  0.835],
       [-0.185, 25.765, -0.24 , ..., -0.315, -0.075,  0.175],
       [-0.34 , -0.24 , 26.635, ..., -0.215, -0.17 ,  0.335],
       ...,
       [-0.155, -0.315, -0.215, ..., 22.785, -0.125,  0.145],
       [-0.19 , -0.075, -0.17 , ..., -0.125, 19.26 ,  0.265],
       [ 0.835,  0.175,  0.335, ...,  0.145,  0.265,  0.   ]])

In [135]:
evalsNew, evctsNew = LA.eig(L_combined)
sortedevalsNew = np.argsort(evalsNew)
UNew = evctsNew[:,sortedevalsNew[1:4]]

In [21]:
from typing import Optional
import Levenshtein
import numpy as np
from sklearn.neighbors import BallTree, DistanceMetric

In [22]:
df_combined["DOB"]

0      1953-10-06
1      1984-08-14
2      1959-10-20
3      1963-03-08
4      1947-02-25
          ...    
160    1949-03-27
161    1989-01-08
162    1949-01-31
163    2020-04-12
164    1981-11-14
Name: DOB, Length: 220, dtype: object

In [324]:
# df_combined['name-dob'] = df_combined['FirstName']+" "+df_combined['Surname']+" "+df_combined['DOB']
# text_data = df_combined['name-dob'].tolist()
# text_data = text_data[55:]

df_combined['name'] = df_combined['FirstName']+" "+df_combined['Surname']
text_data = df_combined['name'].tolist()
text_data = text_data[3300:]

lon_data = df_combined["Longitude"].tolist()
lon_data = lon_data[3300:]

lat_data = df_combined["Latitude"].tolist()
lat_data = lat_data[3300:]

dates_data = df_combined['DOB'].tolist()
dates_data = dates_data[3300:]

age_data = df_combined['P_AGE'].tolist()
age_data = age_data[3300:]


In [313]:
df_combined[df_combined["P_ID"] == "AUT_12013_1"]

,P_num,P_ID,P_AGE,P_GENDER,HH_ISO,Surname,FirstName,DOB,City,Longitude,Latitude,normalized_timestamp,GENDER_BIN,name
99,28446,AUT_12013_1,47,M,AUT,Mathis,Joshua,1974-08-06,Salzburg,13.004300,47.793301,-0.257761,1,Joshua Mathis
99,28446,AUT_12013_1,47,M,AUT,aMthis,Jsohua,1974-08-06,Salzburg,13.157057,46.056543,-0.257761,1,Jsohua aMthis
33099,28446,AUT_12013_1,47,M,AUT,Matihs,Johsua,1974-08-06,Salzburg,12.361772,47.252530,-0.257761,1,Johsua Matihs
66099,28446,AUT_12013_1,47,M,AUT,Mtahis,Johuas,1974-08-06,Salzburg,13.209605,49.044076,-0.257761,1,Johuas Mtahis


In [325]:
df_combined[df_combined["name"] == "Geoffrey Brown"]

,P_num,P_ID,P_AGE,P_GENDER,HH_ISO,Surname,FirstName,DOB,City,Longitude,Latitude,normalized_timestamp,GENDER_BIN,name
3004,120149,BEL_19853_2,53,M,BEL,Brown,Geoffrey,1968-09-01,Brussels,4.484440,50.901402,-0.316742,1,Geoffrey Brown
36004,120149,BEL_19853_2,53,M,BEL,Brown,Geoffrey,1968-09-01,Brussels,4.684468,51.819333,-0.316742,1,Geoffrey Brown


In [307]:


def text_to_vector2(text: str, idx: Optional[int] = None) -> np.ndarray:
    lat = lat_data[idx]  # Get latitude from lat_data
    lon = lon_data[idx]  # Get longitude from lon_data
    dates = dates_data[idx]
    age = age_data[idx]
    if idx is None:
        idx = len(texts)
        texts.append(text)
    return np.array([idx, lat, lon, age], dtype=np.float64)

# def text_to_vector2(text: str, idx: int) -> np.ndarray:
#     lat = lat_data[idx]  # Get latitude from lat_data
#     lon = lon_data[idx]  # Get longitude from lon_data
#     return np.array([idx, lat, lon], dtype=np.float64)

def text_to_vector(text: str, dates: str, lat: float, lon: float, age: int, idx: Optional[int] = None) -> np.ndarray:
    if idx is None:
        idx = len(texts)
        texts.append(text)
        dates_data.append(dates)
        lat_data.append(lat)
        lon_data.append(lon)
        age_data.append(age)
    return np.array([idx, lat, lon, age], dtype=np.float64)

def vector_to_text(idx: np.ndarray) -> str:
    return texts[int(idx[0])]

def dates_to_text(idx: np.ndarray) -> str:
    return dates_data[int(idx[0])]

def get_age(idx: np.ndarray) -> tuple:
    return age_data[int(idx[0])]
 
def fuzzrto(x: np.ndarray, y: np.ndarray) -> int:
    print("X: ", x)
    print("y: ", y)
    x, y = map(vector_to_text, (x, y))
    print("After Mapping")
    print("X: ", x)
    print("y: ", y)
    return (1/fuzz.ratio(x, y))

def haversine(lat1: np.ndarray, lon1: np.ndarray, lat2: np.ndarray, lon2: np.ndarray):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = radians(lat1), radians(lon1)
    lat2, lon2 = radians(lat2), radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

def vector_to_location(idx: np.ndarray) -> tuple:
    return lat_data[int(idx[0])], lon_data[int(idx[0])]

def get_location(idx: np.ndarray) -> tuple:
    return vector_to_location(idx)
 

def combined_distance(x: np.ndarray, y: np.ndarray):
#     print(x)
#     print(y)

    text1 = vector_to_text(x)
    text2 = vector_to_text(y)
    
    date1 = dates_to_text(x)
    date2 = dates_to_text(y)

    text_distance = (100 - fuzz.ratio(text1, text2))/100
    date_text_distance = (100 - fuzz.ratio(date1, date2))/100
    
    query_location = get_location(x)
    data_location = get_location(y)
    location_distance = haversine(query_location[0], query_location[1], data_location[0], data_location[1])
    
    age1 = get_age(x)
    age2 = get_age(y)
    
    age_distance = (abs(age1-age2)-0)/100
#     print("Text Distance: ", text_distance)
#     print("Haversine Distance: ", location_distance)
#     print("Combined Distance: ", text_distance + location_distance)
    max_dist = 3723.221820996304
    min_dist = 0.0
    location_distance = (location_distance-min_dist)/(max_dist-min_dist)
    vector = [date_text_distance, text_distance, location_distance, age_distance]
    combined_distance = np.linalg.norm(vector)
    return combined_distance



def create_query_vector(query, query_lat, query_lon):
    x = text_to_vector(query)  # Create the text-based vector
    # Append query-specific latitude and longitude
    x = np.append(x, [query_lat, query_lon])
    
def create_query_vector2(query, idx, query_lat, query_lon):
    x = text_to_vector(query, idx)  # Create the text-based vector
    x = np.append(x, [query_lat, query_lon])
    return x
                  

# print(X)
# print(x)



In [326]:
texts = 0
text = 0
texts = text_data

query = "Jofrey Brown"
DOB = "1963-09-07"
query_lat = 51.819333 
query_lon = 4.684468  
ages = 50


dates = DOB
knn = 20

                  
X = [text_to_vector2(text,idx) for idx, text in enumerate(texts)]
print("Done X")
x = [text_to_vector(query, dates, query_lat, query_lon, ages)]
print("Done x")

# X = [text_to_vector(text, lat, lon, idx) for idx, text in enumerate(texts), lon in enumerate(lon_data), lat in enumerate(lat_data)]
# x = [text_to_vector(query)]


tree = BallTree(X, metric=DistanceMetric.get_metric('pyfunc', func=lambda x, y: combined_distance(x, y)))
(dists,), (idxs,) = tree.query(x, k=knn)
for dist, idx in zip(dists, idxs):
    print(f'{texts[idx]}, {dates_data[idx]}, {lat_data[idx]}, {lon_data[idx]}, {age_data[idx]}, {int(dist)}')

Done X
Done x


/Users/muhammadfathifadlian/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_distance_metric.py:10: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  warnings.warn(


Geoffrey Brown, 1968-09-01, 51.81933334180265, 4.684467954147478, 53, 0
Gofefrey Bronw, 1968-09-01, 51.862765859665046, 4.355219885802697, 53, 0
Geoffrey Borwn, 1968-09-02, 52.33880841658984, 4.356329556521897, 53, 0
eKrry Brown, 1983-09-02, 54.14174916955302, -4.610613051372055, 38, 0
Kerry Brown, 1983-09-02, 55.87189865, -4.433060169, 38, 0
Connor Brown, 1973-09-19, 50.459202000000005, 4.4538199999999994, 48, 0
Kerry Brown, 1983-09-02, 58.0392529111064, -4.290501096494171, 38, 0
Connor Brown, 1973-09-19, 48.38825358304539, 4.252162979392125, 48, 0
Gareth Brown, 1976-04-08, 51.87469864, -0.368333012, 45, 0
Gareth Brown, 1976-04-08, 51.2543451739801, -0.3741116471241193, 45, 0
Gareth Brown, 1976-04-08, 53.06716520318569, -0.37989256461437054, 45, 0
Keith Brown, 1953-01-07, 51.87469864, -0.368333012, 68, 0
Eric Brown, 1983-02-20, 50.459202000000005, 4.4538199999999994, 38, 0
eKith Brown, 1953-01-07, 49.91078497816185, -0.35064238633538136, 68, 0
Geoffrey Freshour, 1973-09-30, 52.308601,

In [213]:
def haversine_try(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = radians(lat1), radians(lon1)
    lat2, lon2 = radians(lat2), radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [221]:
dist_list = []
for i in range(df_combined.shape[0]):
    for j in range(df_combined.shape[0]):
        dist = haversine_try(df_combined["Latitude"][i], df_combined["Longitude"][i], df_combined["Latitude"][j], df_combined["Longitude"][j])
        dist_list.append(dist)

In [270]:
vector = [3,4]
combined_distance = np.linalg.norm(vector)
combined_distance

5.0